In [34]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance

pd.set_option('display.max_rows', 500)

### Create inputs

In [35]:
objects = ['p', 's', 'g', 'c', 't']
# p = plates (3x), s = silverware (3 forks), g = glasses (3), c = cheese, t = teaspoon

In [36]:
coordinates = {'p': (0,1),
              's': (1,3),
              'g': (0,1),
               'c': (0,0),
               't': (1,3),
              'start': (0,1),
              'table': (1,2)}

### Populate dataframe

In [37]:
def fill_dataframe(objects, objects_at_once=2):
    df = pd.DataFrame(columns=['from_node', 'to_node', 'obj_at_once', 'dist', 'weight_new'])
    # Generate list of combinations w/o duplicates
    nodes = list(itertools.chain(*[itertools.combinations(objects, i+1) for i in range(len(objects))]))
    nodes = [''.join(node) for node in nodes]
    
    node_list = []
    for x in range(0, len(nodes)):
        if len(nodes[x]) <= 2:
            df.loc[x, 'from_node'] = 'z'
            df.loc[x, 'to_node'] = nodes[x]
            df.loc[x, 'obj_at_once'] = len(nodes[x])
    
        for y in range(x, len(nodes)):
            if 1 <= (len(nodes[y]) - len(nodes[x])) <= 2:
                if all(x in nodes[y] for x in nodes[x]):
                    node_list.append((nodes[y], nodes[x]))
    
    for tup in node_list:
        df = df.append({'from_node': tup[1], 'to_node': tup[0], 'obj_at_once': len(tup[0]) - len(tup[1])}, ignore_index=True)
    
    # No constraints for how many objects at once
    if objects_at_once == 2:
        df = df
    elif objects_at_once == 1:
        # Add constraint: Only 1 object at once
        df = df.loc[df['obj_at_once'] == 1]

    # Reset index
    df = df.reset_index(drop=True)
    
    return df

In [23]:
data = fill_dataframe(objects, objects_at_once=1)

### Calculate distances

In [38]:
def calculate_distances(data):
    for row in range(0, len(data)):
        # Distance = start -> obj 1 -> table
        if data['from_node'][row] == 'z' and len(data['to_node'][row]) == 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row]], 
                        coordinates['table'])
                    )
    
        # Distance = start -> obj 1 -> obj 2 -> table
        elif data['from_node'][row] == 'z' and len(data['to_node'][row]) != 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row][0]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][0]], 
                        coordinates[data['to_node'][row][1]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][1]], 
                        coordinates['table'])
                    )
    # Distance = table -> obj 1 -> (obj 2) -> table
        else:
            # Get difference between sequences (from_node, to_node)
            diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
            if len(diff) == 1:
                data.loc[row, 'dist'] = distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) * 2
        
            elif len(diff) == 2:
                data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) +
                    distance.euclidean(
                        coordinates[diff[0]], 
                        coordinates[diff[1]]) +
                    distance.euclidean(
                        coordinates[diff[1]], 
                        coordinates['table'])
                    )
    return data

In [343]:
#data = calculate_distances(data)
#data

In [39]:
c1 = {'p': 1.2,
    's': 1.2,
    'g': 1.2,
    'c': 1.2,
    't': 1.2}

In [40]:
k1 = {'p': 0.95,
    's': 1.0,
    'g': 1.0,
    'c': 1.0,
    't': 1.0}

In [41]:
c0 = {'p': 1.0,
    's': 1.0,
    'g': 1.0,
    'c': 1.0,
    't': 1.0}

k0 = {'p': 1.0,
    's': 1.0,
    'g': 1.0,
    'c': 1.0,
    't': 1.0}

In [42]:
def calculate_edge_weights_params(data, objects, c, k):
    # Reset weights according to weight parameters
    for row in range(0, len(data)):
        diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
        for obj in objects:
            if len(diff) == 1 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                
            elif len(diff) == 2 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                data.loc[row, 'weight_new'] = (data['weight_new'][row] ** k[diff[1]]) * c[diff[1]]
                
    return data

In [12]:
#all(data['weight_new'] == data2['weight_new'])

### Dijkstra + graph classes

In [43]:
class Node:
    def __init__(self, label):
        self.label = label

class Edge:
    def __init__(self, to_node, length):
        self.to_node = to_node
        self.length = length


class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = dict()

    def add_node(self, node):
        self.nodes.add(node)

    def add_edge(self, from_node, to_node, length):
        edge = Edge(to_node, length)
        if from_node.label in self.edges:
            from_node_edges = self.edges[from_node.label]
        else:
            self.edges[from_node.label] = dict()
            from_node_edges = self.edges[from_node.label]
        from_node_edges[to_node.label] = edge


def min_dist(q, dist):
    """
    Returns the node with the smallest distance in q.
    Implemented to keep the main algorithm clean.
    """
    min_node = None
    for node in q:
        if min_node == None:
            min_node = node
        elif dist[node] < dist[min_node]:
            min_node = node

    return min_node

def dijkstra(graph, source):
    q = set()
    dist = {}
    prev = {}

    for v in graph.nodes:           # initialization
        dist[v] = float('inf')      # unknown distance from source to v
        prev[v] = float('inf')      # previous node in optimal path from source
        q.add(v)                    # all nodes initially in q (unvisited nodes)

    # distance from source to source
    dist[source] = 0

    while q:
        # node with the least distance selected first
        u = min_dist(q, dist)

        q.remove(u)

        if u.label in graph.edges:
            for _, v in graph.edges[u.label].items():
                alt = dist[u] + v.length
                if alt < dist[v.to_node]:
                    # a shorter path to v has been found
                    dist[v.to_node] = alt
                    prev[v.to_node] = u

    return dist, prev


def to_array(prev, from_node):
    """Creates an ordered list of labels as a route."""
    previous_node = prev[from_node]
    route = [from_node.label]
    while previous_node != float('inf'):
        route.append(previous_node.label)
        temp = previous_node
        previous_node = prev[temp]

    route.reverse()
    return route

### Create nodes + edges

In [44]:
def create_nodes_edges(graph, data):
    # Create sorted set of nodes from to_nodes and from_nodes
    nodes = sorted(pd.unique(data[['from_node', 'to_node']].values.ravel()))

    # Create nodes from set
    for x in range(0, len(nodes)):
        globals()[nodes[x]] = Node(nodes[x])    # Create node as global variable
        graph.add_node(globals()[nodes[x]])     # Add node to graph

    # Add edge for row in csv -> from_node, to_node, weight
    for row in range(0, len(data)):
        to_node = globals()[data['to_node'][row]]
        from_node = globals()[data['from_node'][row]]
        graph.add_edge(from_node, to_node, data['weight_new'][row])
    
    return graph

In [341]:
#create_nodes_edges(graph, data)

### Print function

In [45]:
def print_result(dist, prev, start, target):
    print("The quickest path from {} to {} is [{}] with a distance of {}".format(
        start.label,
        target.label,
        " -> ".join(to_array(prev, target)),
        str(round(dist[target], 2))
        )
    )

### Define main function

In [46]:
def main():
    graph = Graph()
    data = fill_dataframe(objects, objects_at_once=1)
    data = calculate_distances(data)
    data = calculate_edge_weights_params(data, objects, c0, k0)
    create_nodes_edges(graph, data)
    dist, prev = dijkstra(graph, z)
    print_result(dist, prev, z, psgct)

In [47]:
main()

The quickest path from z to psgct is [z -> p -> pt -> pst -> psgt -> psgct] with a distance of 12.71


In [51]:
data2 = fill_dataframe(objects, objects_at_once=1)
data2 = calculate_distances(data2)
data2 = calculate_edge_weights_params(data2, objects, c0, k1)
data2

from_node to_node obj_at_once     dist weight_new
0          z       p           1  1.41421    1.38992
1          z       s           1  3.23607    3.23607
2          z       g           1  1.41421    1.41421
3          z       c           1  3.23607    3.23607
4          z       t           1  3.23607    3.23607
5          p      ps           1        2          2
6          p      pg           1  2.82843    2.82843
7          p      pc           1  4.47214    4.47214
8          p      pt           1        2          2
9          s      ps           1  2.82843    2.68515
10         s      sg           1  2.82843    2.82843
11         s      sc           1  4.47214    4.47214
12         s      st           1        2          2
13         g      pg           1  2.82843    2.68515
14         g      sg           1        2          2
15         g      gc           1  4.47214    4.47214
16         g      gt           1        2          2
17         c      pc           1  2.82843    2.68515
18         c      sc           1        2          2
19         c      gc           1  2.82843    2.82843
20         c      ct           1        2          2
21         t      pt           1  2.82843    2.68515
22         t      st           1        2          2
23         t      gt           1  2.82843    2.82843
24         t      ct           1  4.47214    4.47214
25        ps     psg           1  2.82843    2.82843
26        ps     psc           1  4.47214    4.47214
27        ps     pst           1        2          2
28        pg     psg           1        2          2
29        pg     pgc           1  4.47214    4.47214
30        pg     pgt           1        2          2
31        pc     psc           1        2          2
32        pc     pgc           1  2.82843    2.82843
33        pc     pct           1        2          2
34        pt     pst           1        2          2
35        pt     pgt           1  2.82843    2.82843
36        pt     pct           1  4.47214    4.47214
37        sg     psg           1  2.82843    2.68515
38        sg     sgc           1  4.47214    4.47214
39        sg     sgt           1        2          2
40        sc     psc           1  2.82843    2.68515
41        sc     sgc           1  2.82843    2.82843
42        sc     sct           1        2          2
43        st     pst           1  2.82843    2.68515
44        st     sgt           1  2.82843    2.82843
45        st     sct           1  4.47214    4.47214
46        gc     pgc           1  2.82843    2.68515
47        gc     sgc           1        2          2
48        gc     gct           1        2          2
49        gt     pgt           1  2.82843    2.68515
50        gt     sgt           1        2          2
51        gt     gct           1  4.47214    4.47214
52        ct     pct           1  2.82843    2.68515
53        ct     sct           1        2          2
54        ct     gct           1  2.82843    2.82843
55       psg    psgc           1  4.47214    4.47214
56       psg    psgt           1        2          2
57       psc    psgc           1  2.82843    2.82843
58       psc    psct           1        2          2
59       pst    psgt           1  2.82843    2.82843
60       pst    psct           1  4.47214    4.47214
61       pgc    psgc           1        2          2
62       pgc    pgct           1        2          2
63       pgt    psgt           1        2          2
64       pgt    pgct           1  4.47214    4.47214
65       pct    psct           1        2          2
66       pct    pgct           1  2.82843    2.82843
67       sgc    psgc           1  2.82843    2.68515
68       sgc    sgct           1        2          2
69       sgt    psgt           1  2.82843    2.68515
70       sgt    sgct           1  4.47214    4.47214
71       sct    psct           1  2.82843    2.68515
72       sct    sgct           1  2.82843    2.82843
73       gct    pgct           1  2.82843    2.68515
74       gct    sgct        